In [1]:
import pandas as pd
import pickle
from IPython.core.display import HTML
HTML("<style>.container { width:100% !important; }</style>")

In [15]:
curr_dir = !pwd
domains_path = curr_dir[0]+"/../3.parse_HMMER/hmm_domains/pfam-v30/"
canonic_prot_path = curr_dir[0]+"/../4.parse_Uniprot/domains_canonic_prot/pfam-v30/"

In [8]:
domains_filenames = !ls $domains_path

In [99]:
def count_domain_instances(domain_gene_table, count_overlaps=True):
    
    if (count_overlaps):
        return domain_gene_table.shape[0]
    
    else:
        instance_counter = 0
        last_target_end = 0

        for i, row in domain_gene_table.iterrows():
            curr_target_start = int(row["TargetStart"])
            curr_target_end = int(row["TargetEnd"])
            if (curr_target_start > last_target_end):
                instance_counter += 1
                last_target_end = curr_target_end
            #If the instance overlpas the previous one
            else: 
                #Updating to the smaller traget end
                if (curr_target_end > last_target_end):
                    last_target_end = curr_target_end
                #Continue without incrememnting the counter
                continue 
        
        return instance_counter

In [102]:
domains_stats = {}

for dom_filename in domains_filenames:
    curr_domain_stats = []
    
    domain_sym = dom_filename[:dom_filename.find(".")]
    domain_data = pd.read_csv(domains_path+dom_filename, sep='\t', index_col=0)
    domain_ens_genes = (domain_data["gene"]).unique()
    
    with open(canonic_prot_path+domain_sym+"_canonic_prot.pik", 'rb') as handle:
        canonic_protein = pickle.load(handle)
    
    domain_intance_num = 0
    for ens_gene in domain_ens_genes:
        #Filtering the domain data for this gene according to the canonical protein id
        canonic_prot = canonic_protein[ens_gene]
        canonic_prot_t = canonic_prot[:canonic_prot.find(".")] #Trimming the ".#" at the end
        domain_gene_table = domain_data[domain_data["prot"] == canonic_prot]
        
        #Count the number of domain instances in this gene
        domain_gene_table = domain_gene_table.sort_values(by=["TargetStart", "BitScore"], ascending=[True, False])
        gene_intance_num = count_domain_instances(domain_gene_table, count_overlaps=True)
        domain_intance_num += gene_intance_num
    
    #Saving domains stats:
    curr_domain_stats.append(len(domain_ens_genes)) # No. of genes
    curr_domain_stats.append(domain_intance_num) #No. of domain instances
    
    #Updating the big dict
    domains_stats[domain_sym] = curr_domain_stats

In [147]:
domains_stats_df = pd.DataFrame.from_dict(domains_stats,orient='index')
domains_stats_df.columns = ["genes", "instances"]
domains_stats_df = domains_stats_df.sort_values(by=["instances", "genes"], ascending=[False, False])
domains_stats_df.to_csv(curr_dir[0]+"/domains_stats_df.csv", sep='\t')

In [149]:
#Filter domains according to number of instances
instance_threshold = 50
filtered_domains_df = domains_stats_df[domains_stats_df["instances"] > instance_threshold]

#Save to file
filtered_domains_df.to_csv(curr_dir[0]+"/filtered50_domains_df.csv", sep='\t')